In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from joblib import Parallel, delayed

In [ ]:
!apt install -y ffmpeg

In [ ]:
AUDIO_PATH = '../input/birdclef-2022/train_audio'
CLASSES = sorted(os.listdir(AUDIO_PATH))
NUM_CLASSES = len(CLASSES)

In [ ]:
class AudioParams:
    """
    Parameters used for the audio data
    """
    sr = 22050
    duration = 5
    n_mels = 224
    fmin = 20
    fmax = 16000

In [ ]:
def crop_or_pad(y, length, sr, train=True, probs=None):
    """
    Crops an array to a chosen length
    Arguments:
        y {1D np array} -- Array to crop
        length {int} -- Length of the crop
        sr {int} -- Sampling rate
    Keyword Arguments:
        train {bool} -- Whether we are at train time. If so, crop randomly, else return the beginning of y (default: {True})
        probs {None or numpy array} -- Probabilities to use to chose where to crop (default: {None})
    Returns:
        1D np array -- Cropped array
    """
    if len(y) <= length:
        y = np.concatenate([y, np.zeros(length - len(y))])
    else:
        if not train:
            start = 0
        elif probs is None:
            start = np.random.randint(len(y) - length)
        else:
            start = (
                    np.random.choice(np.arange(len(probs)), p=probs) + np.random.random()
            )
            start = int(sr * (start))

        y = y[start: start + length]

    return y.astype(np.float32)

In [ ]:
def preprocess_soundfiles(path, data=mydict):
    for file in os.listdir(os.path.join(AUDIO_PATH, path)):
        loc = os.path.join(AUDIO_PATH, path)
        bird_call, _ = librosa.load(os.path.join(loc, file), mono=True)
        segment = crop_or_pad(bird_call, AudioParams.duration * AudioParams.sr, sr=AudioParams.sr, train=True, probs=None)
        mfcc = librosa.feature.mfcc(y=segment, sr=AudioParams.sr, hop_length=1024, n_mfcc=13)
        mfcc = mfcc.T
        data["mfcc"].append(mfcc)
        data["labels"].append(os.path.basename(dirname))        

In [ ]:
mydict = {
        "mfcc": [],
        "labels": []
        }

In [ ]:
for path in os.listdir(AUDIO_PATH):
    for file in os.listdir(os.path.join(AUDIO_PATH, path)):
        loc = os.path.join(AUDIO_PATH, path)
        print(os.path.join(loc, file))

In [ ]:
_ = Parallel(n_jobs=4)(delayed(preprocess_soundfiles)(path) for path in tqdm(os.listdir(AUDIO_PATH)))

In [ ]:
mydict["mfcc"]

In [ ]:
X = np.array(data)
targets = np.array(labels)
y = pd.get_dummies(targets).values

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,random_state=1)